In [85]:
import json
import re
import pandas as pd
from pandas.io.json import json_normalize

#pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 110)

In [86]:
def flattenJSON(json_obj):   
    data = json.loads(json_obj)
    
    def flatten_dict(data):
        for k in list(data):
            if isinstance(data[k], dict):
                child = flatten_dict(data[k])
                for k_ in child:
                    data[k + '.' + k_] = child[k_]
                else:
                    del data[k]
        return data
    
    return flatten_dict(data)

In [87]:
def toConvertableJson(s):    
    def _convertableJson(matchObj):
        if matchObj.group(1):
            return '"'
        elif matchObj.group(2):
            return ':"False"'
        elif matchObj.group(3):
            return ':"True"'
    
    d = flattenJSON(s)
    for i,v in enumerate(d):
        if isinstance(d[v], str) and ('{' in d[v]):
            d[v] = json.loads(re.sub('(\')|(: False)|(: True)',_convertableJson, d[v]))
    return flattenJSON(json.dumps(d))

In [4]:
with open('yelp_dataset/user.json') as f:
    data = [json.loads(line) for line in f]
user = pd.DataFrame(data)
user = user.sample(frac=0.05, replace=False, random_state=1)
user

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
888070,TQnJM4pv9pxwRt64Gj5JfA,Amy,5,2012-11-04 11:58:43,5,0,1,,"Sxnf9uOrAIy7HOM3NYyF-g, dLC-DsQGkmWF--7076CWmA",0,...,0,0,0,0,0,0,0,0,1,0
786620,_I148UPrAgo-SZ9qOkwX3A,Pat,1,2018-10-24 20:15:13,0,0,0,,"WitMA5a9h_1XQAsb34Xtmw, GUsHfw4QmOaVmXvoqWDD1A...",0,...,0,0,0,0,0,0,0,0,0,0
43384,1e4CMKgujUHagjQgv3NVCA,Carlos,11,2014-06-14 18:07:23,1,2,2,,"WQQsl6_9QhqgJY0Zr5gMnQ, wX2ETi3RJMb6pRmN7Y3jSA...",0,...,0,0,0,0,0,0,0,0,0,0
12854,F0nR2cEBLSZtBb2hlxnpoA,May,159,2006-06-22 04:29:33,173,59,74,"2015,2016,2017","InVoTvQufmEmcVsSjUrtEw, mOKi4ysh1CK2rLu8pKwBAQ...",12,...,2,1,0,0,2,2,7,7,4,1
394483,B1SdzXmYZvK7QZwZ1g_XCg,Marlene,8,2015-06-20 00:57:29,3,1,0,,"hFQ9pR73raKHR031Y_skYw, DV1w2IaGhifIjdKLQReWXw...",0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534886,YGEQoYKCs0nJ05VRAKShSg,Zelda,37,2012-06-13 03:22:44,51,14,9,,"8JjDxdld-IZnKIDifXWuww, G_eVitxNoQvdgT6qZG9nmA...",1,...,1,0,0,0,1,1,1,1,0,0
1471474,ZBWFfWR4CD9_m-A3kmZJ3Q,Matthew,1,2016-10-21 13:47:44,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
667752,6ynYD4jt071290lO96ywhQ,Christopher,3,2016-05-27 04:05:03,1,0,0,,"IPzUXp2SSgyHyCyrDo9vEA, EsuZd0tiQ5ABk-Mgb4oHzQ...",0,...,0,0,0,0,0,0,0,0,0,0
303412,aeMjXQ-aBJ1h1JnaA8pR1Q,Mnementh,3,2010-07-13 20:02:46,1,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
with open('yelp_dataset/review.json') as f:
    data = [json.loads(line) for line in f]
review = pd.DataFrame(data)
review = review.sample(frac=0.05, replace=False, random_state=1)
review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
4634629,xw37uw9b2wFwVRLXSCtgNg,tIau2r-ox1LhC38wXX6oew,IUoj0ORP1VAV9xx4dmHP9g,5.0,0,0,0,I love this location and have been a customer ...,2018-09-07 20:41:56
4349124,jKxK47dz6D-ZpKqmd-oTeA,q0lV9MoRYDhyoLAQ9X91iA,hihud--QRriCYZw1zZvW4g,1.0,0,0,0,I think that the food is not at all authentic ...,2018-01-03 03:57:16
6056428,boaG5qtsdwR3KBwQbvnfAg,kUE8PMxx2_HTT32T16Z8Aw,zT3IxSfolhu4qU0tV0pTjg,2.0,1,0,0,If you schedule your appointment late in the a...,2018-06-27 01:40:52
1713906,ah5BuOXlUwUIfSYHPa9Syg,0wOrT_OF0QN7bWlbgCc2ww,em__JLa6eEyYXzMbbVEahQ,5.0,0,0,0,Wow! What can I say about Shauna? Plenty of th...,2013-07-08 06:53:20
3794033,TB2cTa0ji3JaYbInQN_nqQ,oKhpMYvIntENZ5UzuoZWUg,1KhBzrqmU164bss64jR5uA,4.0,0,0,0,Fun night of karaoke. Food good...lady barten...,2016-03-16 05:56:01
...,...,...,...,...,...,...,...,...,...
4491823,g9ahL53bo3ATOdsxfr9cgw,ZkltFQ9FEM75pFS6dmtl1w,CO8MpD1bSkBPArXKBKkIfA,5.0,0,0,0,So happy they have this in Arizona now! Taste ...,2015-11-01 19:46:24
6203458,oucYbPoOCSRjk4Mv-EWaQA,x8e1Cc1lh835Q1CpHAD0RQ,UT22ExffDZjB1tzBDwSWfA,5.0,0,0,0,The service was awesome! Thank you Parker! I h...,2016-04-03 03:28:22
1881208,vvx62GHUKs5d6eO7tuca8A,00qgu1BIDCUtS3zC7bit9g,YzWB7zn0b40Ijw1tkBnt0A,1.0,5,1,0,I didn't know they were part of the Bell autom...,2015-06-29 20:04:42
2705634,K4uPpx-J284iAAI208VViQ,NOEeBryAxOWFJoTkmbkckw,wVLvEcK05JGhw_EmdTpF9Q,5.0,2,1,1,I was so happy with my experience today in the...,2016-10-28 04:51:17


In [28]:
with open('yelp_dataset/business.json') as f:
    data = [toConvertableJson(line) for line in f]
business = pd.DataFrame(data)
#business = business.sample(frac=0.05, replace=False, random_state=1)
business.drop(['hours','attributes'], axis=1, inplace=True) # null

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours,attributes.GoodForKids,attributes.RestaurantsReservations,attributes.Caters,attributes.NoiseLevel,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange2,attributes.OutdoorSeating,attributes.BikeParking,attributes.HasTV,attributes.WiFi,attributes.Alcohol,attributes.RestaurantsAttire,attributes.RestaurantsGoodForGroups,attributes.RestaurantsDelivery,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.GoodForMeal.dessert,attributes.GoodForMeal.latenight,attributes.GoodForMeal.lunch,attributes.GoodForMeal.dinner,attributes.GoodForMeal.brunch,attributes.GoodForMeal.breakfast,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet,attributes.Ambience.romantic,attributes.Ambience.intimate,attributes.Ambience.classy,attributes.Ambience.hipster,attributes.Ambience.divey,attributes.Ambience.touristy,attributes.Ambience.trendy,attributes.Ambience.upscale,attributes.Ambience.casual,attributes.BusinessAcceptsCreditCards,attributes,attributes.BusinessAcceptsBitcoin,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes.GoodForDancing,attributes.CoatCheck,attributes.HappyHour,attributes.Music.dj,attributes.Music.background_music,attributes.Music.no_music,attributes.Music.jukebox,attributes.Music.live,attributes.Music.video,attributes.Music.karaoke,attributes.BestNights.monday,attributes.BestNights.tuesday,attributes.BestNights.friday,attributes.BestNights.wednesday,attributes.BestNights.thursday,attributes.BestNights.sunday,attributes.BestNights.saturday,attributes.Ambience,attributes.BusinessParking,attributes.WheelchairAccessible,attributes.DogsAllowed,attributes.BYOBCorkage,attributes.DriveThru,attributes.Smoking,attributes.AgesAllowed,attributes.HairSpecializesIn.straightperms,attributes.HairSpecializesIn.coloring,attributes.HairSpecializesIn.extensions,attributes.HairSpecializesIn.africanamerican,attributes.HairSpecializesIn.curly,attributes.HairSpecializesIn.kids,attributes.HairSpecializesIn.perms,attributes.HairSpecializesIn.asian,attributes.Corkage,attributes.BYOB,attributes.GoodForMeal,attributes.Music,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.gluten-free,attributes.DietaryRestrictions.vegan,attributes.DietaryRestrictions.kosher,attributes.DietaryRestrictions.halal,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.BestNights,attributes.HairSpecializesIn,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,"Golf, Active Life",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"Specialty Food, Restaurants, Dim Sum, Imported...",NaN,True,True,True,u'loud',True,True,2,False,False,False,u'no',u'full_bar',u'casual',True,False,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-1:0,9:0-1:0,9:0-0:0,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gnKjw

In [67]:
business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.GoodForKids,attributes.RestaurantsReservations,attributes.Caters,attributes.NoiseLevel,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange2,attributes.OutdoorSeating,attributes.BikeParking,attributes.HasTV,attributes.WiFi,attributes.Alcohol,attributes.RestaurantsAttire,attributes.RestaurantsGoodForGroups,attributes.RestaurantsDelivery,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.GoodForMeal.dessert,attributes.GoodForMeal.latenight,attributes.GoodForMeal.lunch,attributes.GoodForMeal.dinner,attributes.GoodForMeal.brunch,attributes.GoodForMeal.breakfast,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet,attributes.Ambience.romantic,attributes.Ambience.intimate,attributes.Ambience.classy,attributes.Ambience.hipster,attributes.Ambience.divey,attributes.Ambience.touristy,attributes.Ambience.trendy,attributes.Ambience.upscale,attributes.Ambience.casual,attributes.BusinessAcceptsCreditCards,attributes.BusinessAcceptsBitcoin,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes.GoodForDancing,attributes.CoatCheck,attributes.HappyHour,attributes.Music.dj,attributes.Music.background_music,attributes.Music.no_music,attributes.Music.jukebox,attributes.Music.live,attributes.Music.video,attributes.Music.karaoke,attributes.BestNights.monday,attributes.BestNights.tuesday,attributes.BestNights.friday,attributes.BestNights.wednesday,attributes.BestNights.thursday,attributes.BestNights.sunday,attributes.BestNights.saturday,attributes.Ambience,attributes.BusinessParking,attributes.WheelchairAccessible,attributes.DogsAllowed,attributes.BYOBCorkage,attributes.DriveThru,attributes.Smoking,attributes.AgesAllowed,attributes.HairSpecializesIn.straightperms,attributes.HairSpecializesIn.coloring,attributes.HairSpecializesIn.extensions,attributes.HairSpecializesIn.africanamerican,attributes.HairSpecializesIn.curly,attributes.HairSpecializesIn.kids,attributes.HairSpecializesIn.perms,attributes.HairSpecializesIn.asian,attributes.Corkage,attributes.BYOB,attributes.GoodForMeal,attributes.Music,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.gluten-free,attributes.DietaryRestrictions.vegan,attributes.DietaryRestrictions.kosher,attributes.DietaryRestrictions.halal,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.BestNights,attributes.HairSpecializesIn,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,"Golf, Active Life",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"Specialty Food, Restaurants, Dim Sum, Imported...",True,True,True,u'loud',True,True,2,False,False,False,u'no',u'full_bar',u'casual',True,False,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-1:0,9:0-1:0,9:0-0:0,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanes

In [96]:
cat = []
[cat.extend(n.split(',')) for n in business['categories'] if n is not None ]
cat = [n.strip() for n in cat]
cat = [(n, cat.count(n)) for n in set(cat)]
cat.sort(key=lambda e:e[1], reverse=True)
cat

[('Restaurants', 59371),
 ('Shopping', 31878),
 ('Food', 29989),
 ('Home Services', 19729),
 ('Beauty & Spas', 19370),
 ('Health & Medical', 17171),
 ('Local Services', 13932),
 ('Automotive', 13203),
 ('Nightlife', 13095),
 ('Bars', 11341),
 ('Event Planning & Services', 10371),
 ('Active Life', 9521),
 ('Fashion', 7798),
 ('Sandwiches', 7332),
 ('Coffee & Tea', 7321),
 ('Fast Food', 7257),
 ('American (Traditional)', 7107),
 ('Hair Salons', 6955),
 ('Pizza', 6804),
 ('Home & Garden', 6489),
 ('Arts & Entertainment', 6304),
 ('Professional Services', 6276),
 ('Auto Repair', 6140),
 ('Hotels & Travel', 6033),
 ('Doctors', 5867),
 ('Real Estate', 5677),
 ('Burgers', 5404),
 ('Breakfast & Brunch', 5381),
 ('Nail Salons', 5043),
 ('Specialty Food', 4883),
 ('American (New)', 4882),
 ('Italian', 4716),
 ('Fitness & Instruction', 4646),
 ('Mexican', 4618),
 ('Chinese', 4428),
 ('Pets', 4111),
 ('Hair Removal', 4002),
 ('Bakeries', 3711),
 ('Grocery', 3609),
 ('Dentists', 3540),
 ('Skin Care

### check null

In [32]:
s = pd.DataFrame({'unique_count':business.nunique(),'null_count':business.isnull().sum()})
#s[s['unique_count']==0]
s

,unique_count,null_count
business_id,192609,0
name,145046,0
address,151977,0
city,1204,0
state,36,0
...,...,...
attributes.BestNights,1,192602
attributes.HairSpecializesIn,1,192578
attributes.Open24Hours,2,192596
attributes.DietaryRestrictions,1,192602


In [7]:
with open('yelp_dataset/checkin.json') as f:
    data = [json.loads(line) for line in f]
checkin = pd.DataFrame(data)
checkin = checkin.sample(frac=0.05, replace=False, random_state=1)
checkin

,business_id,date
156756,xx9hljOh-M5NYojRltqpBw,"2010-07-01 02:01:40, 2010-07-13 02:44:44, 2010..."
118887,iuLcQBpGT_r4DTIHGcMUzA,"2013-01-13 22:44:10, 2014-02-02 18:59:13, 2014..."
115955,hkazOWo9uqGemkRww7YGyw,2013-03-24 17:45:11
76428,T7zfYeaMM7G5hratsil6Zg,"2010-10-08 21:59:35, 2011-10-21 19:05:58, 2011..."
33136,C5MFLsL149dfq0dAT-8STw,"2013-11-16 20:42:30, 2013-11-26 19:35:50, 2014..."
...,...,...
107587,eTBNs0QYa9fUOrNsZEllAA,"2012-03-03 00:45:59, 2012-11-12 22:14:00, 2012..."
68419,PxH02Eu2Z4MUycBLU80D8g,"2010-07-24 17:55:48, 2010-08-25 10:09:47, 2011..."
147914,uSB0fH55lobpipg_S14PLw,"2012-09-15 15:42:46, 2012-12-23 00:34:57, 2013..."
124234,l1Vt6y_0ntMhErLBRgFHFA,"2010-01-23 19:25:51, 2014-03-13 00:31:24, 2014..."


In [8]:
with open('yelp_dataset/tip.json') as f:
    data = [json.loads(line) for line in f]
tip = pd.DataFrame(data)
tip = tip.sample(frac=0.05, replace=False, random_state=1)
tip

,user_id,business_id,text,date,compliment_count
756522,ocCMffVARqrlQJrIB6TW0Q,065c76tt1dXBNmoGBymUgQ,Great hot dogs and cold beer. Go America!!!!,2017-11-24 21:32:31,0
786205,z7cXkDAXlqEi9aCF1p9nAQ,Fv3HvJPQYqfmniTgtIQJzA,"Great coffee, pastries and atmosphere",2017-07-23 03:44:05,0
302727,3tFW6x0PaEIiokZTjKAVqw,Xny0n0s98TpP82sQxfgIMQ,Good comfort food.,2014-07-31 05:59:21,0
162622,SiJbi5-NzyJRPgYVlhAorg,IsoLzudHC50oJLiEWpwV-w,Awesome kale salad,2011-10-02 21:49:43,0
34493,1y1T-gONnhnC68JlR6RNgA,vpoXE7CsEMZNzBMDvkuqHg,"Good flavor, slightly Americanized. Lunch Pan...",2012-10-20 19:41:29,0
...,...,...,...,...,...
208137,Ue_7yUlkEbX4AhnYdUfL7g,yoxtSQ8IXBbRBH4KUKO3eA,Call and make a reservation to assure a spot a...,2016-06-10 23:46:45,0
168931,1Av1yTHSl4pi6MPMLC021A,g8OnV26ywJlZpezdBnOWUQ,big pancake!,2011-02-13 06:45:20,0
310626,vLUHU5KxoB_CzLSHLkwz-g,xJzJpTYC_1k_EE0AWTOPXw,Going to watch The Avengers!,2012-05-05 01:53:40,0
27128,TGk5vjg-xTREPuKa7kEfkQ,H9sg6Vq9m9C9EyBEPypPwg,I couldn't rely on my friends when it came to ...,2013-08-27 17:05:16,0
